In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import *
from PIL import Image
from torchvision import transforms
import torchvision.transforms.functional as tf
import os
from torch.optim import RMSprop
from torchvision.utils import make_grid
from pylab import plt
import Mod.res_block as resblock

In [2]:
tar_path=os.path.abspath('../../dataset/GZSL_for_AWA/Animals_with_Attributes2/')
bat_size=24
img_size=227
gpu=True
tr_epoch=2
worker=2


transform1=transforms.Compose([
    transforms.Resize((img_size,img_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3,[0.5]*3)
])


In [3]:
#载入测试集
class SeenData(Dataset):
    def __init__(self,root,transforms=None,train_data=True,label=None):
        #路径
        #self.path=root
        
        if train_data==True:
            #root_label=os.path.abspath('../dataset/Animals_with_Attributes2')
            label_path=[x.path for x in os.scandir(root) if 'trainclass' in x.path]
            word=[]
            for i,file_label in enumerate(label_path):
                f=open(file_label,'r')
                while True:
                    temp_w=f.readline().strip()
                    if not temp_w:
                        break
                    word.append(temp_w)
                f.close()
            
            path_dir=[]
            self.label_list=word
            for key_word in self.label_list:
                [path_dir.append(x.path) for x in os.scandir(root+'/trainingset/') if x.name.endswith(key_word)]
                
            pic_path=[]
            label_t=[]
            for l,train_root in enumerate(path_dir):#对训练集文件夹遍历
                #[pic_path.append(x.path) for x in os.scandir(train_root) if x.name.endswith(".jpg")]#每个文件夹里的图片地址
                for x in os.scandir(train_root):
                    if x.name.endswith(".jpg"):
                        pic_path.append(x.path)
                        label_t.append(l)
                        
            #pic_path=path_dir#图片文件夹目录
            label_np=np.array(label_t)
            label_tensor=torch.from_numpy(label_np).type(torch.LongTensor)
            self.images_files=pic_path #图片目录
            self.transforms=transforms
            self.label=label_tensor
            
        else:
            #self.path#测试机读取
            label_path=[x.path for x in os.scandir(root) if 'trainclass' in x.path]
            word=[]
            for i,file_label in enumerate(label_path):
                f=open(file_label,'r')
                while True:
                    temp_w=f.readline().strip()
                    if not temp_w:
                        break
                    word.append(temp_w)
                f.close()
                
            path_dir=[]
            self.label_list=word
            for key_word in self.label_list:
                [path_dir.append(x.path) for x in os.scandir(root+'/testingset/') if x.name.endswith(key_word)]
                
            pic_path=[]
            label_t=[]
            for l,train_root in enumerate(path_dir):#对训练集文件夹遍历
                #[pic_path.append(x.path) for x in os.scandir(train_root) if x.name.endswith(".jpg")]#每个文件夹里的图片地址
                for x in os.scandir(train_root):
                    if x.name.endswith(".jpg"):
                        pic_path.append(x.path)
                        label_t.append(l)
                        
            #pic_path=path_dir#图片文件夹目录
            label_np=np.array(label_t)
            label_tensor=torch.from_numpy(label_np).type(torch.LongTensor)
            self.images_files=pic_path #图片目录
            self.transforms=transforms
            self.label=label_tensor
            
    def __getitem__(self,index):
        try:
            pic_data=self.transforms(Image.open(self.images_files[index]))
            
        except RuntimeError:
            print(self.images_files[index])
            return self.transforms(Image.open(self.images_files[index-1])),self.label[index]
            
        else:
            #print(self.images_files[index])
            return pic_data,self.label[index]
        
    def __len__(self):
        return len(self.images_files)
        

In [4]:
#模型
class AlexNet(nn.Module):
    def __init__(self, num_classes=1000):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), 256 * 6 * 6)
        x = self.classifier(x)
        return x
    
#网络resnet
def Conv1(in_planes, places, stride=2):
    return nn.Sequential(
        nn.Conv2d(in_channels=in_planes,out_channels=places,kernel_size=7,stride=stride,padding=3, bias=False),
        nn.BatchNorm2d(places),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
    )

class Bottleneck(nn.Module):
    def __init__(self,in_places,places, stride=1,downsampling=False, expansion = 4):
        super(Bottleneck,self).__init__()
        self.expansion = expansion
        self.downsampling = downsampling

        self.bottleneck = nn.Sequential(
            nn.Conv2d(in_channels=in_places,out_channels=places,kernel_size=1,stride=1, bias=False),
            nn.BatchNorm2d(places),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=places, out_channels=places, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(places),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=places, out_channels=places*self.expansion, kernel_size=1, stride=1, bias=False),
            nn.BatchNorm2d(places*self.expansion),
        )

        if self.downsampling:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_channels=in_places, out_channels=places*self.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(places*self.expansion)
            )
        self.relu = nn.ReLU(inplace=True)
    def forward(self, x):
        residual = x
        out = self.bottleneck(x)

        if self.downsampling:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)
        return out
    
class ResNet(nn.Module):
    def __init__(self,blocks, num_classes=1000, expansion = 4):
        super(ResNet,self).__init__()
        self.expansion = expansion

        self.conv1 = Conv1(in_planes = 3, places= 64)

        self.layer1 = self.make_layer(in_places = 64, places= 64, block=blocks[0], stride=1)
        self.layer2 = self.make_layer(in_places = 256,places=128, block=blocks[1], stride=2)
        self.layer3 = self.make_layer(in_places=512,places=256, block=blocks[2], stride=2)
        self.layer4 = self.make_layer(in_places=1024,places=512, block=blocks[3], stride=2)

        #self.avgpool = nn.AvgPool2d(7, stride=1)
        self.avgpool=nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(2048,num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def make_layer(self, in_places, places, block, stride):
        layers = []
        layers.append(Bottleneck(in_places, places,stride, downsampling =True))
        for i in range(1, block):
            layers.append(Bottleneck(places*self.expansion, places))

        return nn.Sequential(*layers)


    def forward(self, x):
        x = self.conv1(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [5]:
#模型读取
#alex=AlexNet(num_classes=40)
#alex.load_state_dict(torch.load('./models/1seenCNN.pkl'))

net=ResNet([3, 4, 23, 3],num_classes=40)
net.load_state_dict(torch.load('./models/800seenResNet.pkl'))

indata=SeenData(tar_path,transform1,train_data=False)#
data_testing=DataLoader(indata,bat_size,shuffle = False,num_workers=worker)
if gpu:
    net.cuda()
    


In [6]:
#测试
error_rate=0
total_sam=0
error_sam=0
for i_batch,batch in enumerate(data_testing):
    data=batch[0]
    tar_label=batch[1]
    zero_tensor=torch.zeros(data.size())
    #print(data.size())
    one_tensor=torch.ones(data.size())
    if gpu:
        data=data.cuda()
        #zero_ten=zero_tensor.cuda()
        #one_ten=one_tensor.cuda()
        tar_label=tar_label.cuda()
    output=net(data)
    #output=alex(data)
    #print(output.cpu())#.argmax(dim=1)
    error_mat=output.cpu().argmax(dim=1)-tar_label.cpu()
    #print(error_mat)
    error_sam=error_mat[error_mat!=0].size()[0]+error_sam#累计错误个数
    total_sam=total_sam+data.size()[0]#样本总个数
    #label_out=alex(data).cpu()#.argmax(dim=1)
    #print(label_out)
    #print(tar_label.cpu())
    print("batch end")
error_rate=error_sam/total_sam
print(error_rate)

batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end
batch end


In [22]:
#网络参数
for para in alex.parameters():
    print(para)

Parameter containing:
tensor([[[[ 1.2352e-01, -6.8222e-02,  3.3419e-02,  ..., -1.1062e-01,
           -2.7035e-01, -2.2427e-01],
          [-9.2817e-02, -7.7265e-02, -4.6631e-02,  ..., -1.4963e-01,
           -2.2933e-01, -7.6633e-02],
          [-1.2257e-01, -2.0544e-01, -1.5108e-01,  ..., -5.0145e-02,
           -1.3678e-01, -1.3915e-01],
          ...,
          [-7.1359e-02, -1.4066e-01, -5.5019e-02,  ...,  8.3426e-03,
           -1.2072e-01, -1.6380e-01],
          [-4.3399e-02, -7.1227e-02, -1.1957e-01,  ..., -5.6895e-02,
           -3.4328e-02, -2.2087e-01],
          [-4.4605e-03,  2.9588e-02, -3.2179e-02,  ..., -3.8138e-02,
           -6.1089e-02, -1.8102e-01]],

         [[ 1.2098e-01,  1.7887e-02,  4.3181e-02,  ..., -1.2687e-01,
           -2.0072e-01, -5.2427e-02],
          [-1.4099e-01, -1.0704e-01, -9.1107e-02,  ..., -1.4023e-01,
           -1.0556e-01, -5.8048e-02],
          [-1.1762e-01, -1.8373e-01, -4.2517e-02,  ...,  5.2762e-02,
            2.8956e-03, -3.3687e-02]